In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Load Data

In [3]:
# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('data/train.csv', delimiter=',', skiprows=1)
X = dataset[:,1:]
print(X)
y = dataset[:, 0]
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[1. 0. 1. ... 7. 6. 9.]


In [4]:
# Normalize the training data
X = X / 255.0  # Normalize pixel values to [0, 1]

In [5]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)  # Class indices should be int64

Define PyTorch Model

In [6]:
model = nn.Sequential(
    nn.Linear(784, 300),
    nn.ReLU(),
    nn.Linear(300, 300),
    nn.ReLU(),
    nn.Linear(300, 10)
)

print(model)

Sequential(
  (0): Linear(in_features=784, out_features=300, bias=True)
  (1): ReLU()
  (2): Linear(in_features=300, out_features=300, bias=True)
  (3): ReLU()
  (4): Linear(in_features=300, out_features=10, bias=True)
)


Define Loss Function and Optimizers

In [7]:
loss_fn = nn.CrossEntropyLoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

Run a Training Loop

In [ ]:
# Run the training loop
n_epochs = 10
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i + batch_size]
        ybatch = y[i:i + batch_size]

        # Forward pass
        y_pred = model(Xbatch)

        # Compute loss
        loss = loss_fn(y_pred, ybatch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Finished epoch {epoch}, latest loss {loss.item()}')

Evaluate the Model

In [ ]:
# Evaluate the model
with torch.no_grad():
    y_pred = model(X)
    predictions = torch.argmax(y_pred, dim=1)
    accuracy = (predictions == y).float().mean()
    print(f"Accuracy: {accuracy.item()}")

Make Predictions

In [ ]:
# Load the test dataset
test_data_path = 'data/test.csv'
test_df = pd.read_csv(test_data_path)

# Prepare the test data
test_X = test_df.values
test_X = test_X / 255.0  # Normalize pixel values to [0, 1]
test_X = torch.tensor(test_X, dtype=torch.float32)

model.eval()

# Make predictions
with torch.no_grad():
    outputs = model(test_X)
    test_predictions = torch.argmax(outputs, dim=1).numpy()

# Create a DataFrame with ImageId and Label
image_ids = np.arange(1, len(test_predictions) + 1)  # Assuming ImageId starts from 1
submission_df = pd.DataFrame({
    'ImageId': image_ids,
    'Label': test_predictions
})

# Save to CSV file
submission_df.to_csv('submission.csv', index=False)

In [ ]:
# Select a few samples to visualize
num_samples = 10
sample_indices = np.random.choice(len(test_X), num_samples, replace=False)
sample_images = test_X[sample_indices].numpy()
sample_predictions = test_predictions[sample_indices]

# Visualize the samples along with their predicted labels
fig, axes = plt.subplots(1, num_samples, figsize=(20, 2))
for i, ax in enumerate(axes):
    ax.imshow(sample_images[i].reshape(28, 28), cmap='gray')
    ax.set_title(f'Pred: {sample_predictions[i]}')
    ax.axis('off')
plt.show()